## Data Ingestion

In [68]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter  # fix: correct package name
from pathlib import Path

In [69]:
def process_pdfs_in_directory(directory_path):
    pdf_loader = DirectoryLoader(
        directory_path,
        glob="**/*.pdf",
        loader_cls=PyMuPDFLoader,
        show_progress=True,
    )
    documents = pdf_loader.load()
    return documents
    # text_splitter = RecursiveCharacterTextSplitter(
    #     chunk_size=100,
    #     chunk_overlap=200,
    # )
    # split_documents = text_splitter.split_documents(documents)
    # print(f"Processed {len(split_documents)} document chunks from PDFs in {directory_path}")
    # return split_documents

documents = process_pdfs_in_directory("../data/pdf")
documents

100%|██████████| 2/2 [00:00<00:00, 28.32it/s]


[Document(metadata={'producer': 'PyPDF2', 'creator': '', 'creationdate': '', 'source': '../data/pdf/NIPS-2017-attention-is-all-you-need-Paper.pdf', 'file_path': '../data/pdf/NIPS-2017-attention-is-all-you-need-Paper.pdf', 'total_pages': 11, 'format': 'PDF 1.3', 'title': 'Attention is All you Need', 'author': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, Illia Polosukhin', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'keywords': '', 'moddate': '2018-02-12T21:22:10-08:00', 'trapped': '', 'modDate': "D:20180212212210-08'00'", 'creationDate': '', 'page': 0}, page_content='Attention Is All You Need\nAshish Vaswani∗\nGoogle Brain\navaswani@google.com\nNoam Shazeer∗\nGoogle Brain\nnoam@google.com\nNiki Parmar∗\nGoogle Research\nnikip@google.com\nJakob Uszkoreit∗\nGoogle Research\nusz@google.com\nLlion Jones∗\nGoogle Research\nllion@google.com\nAidan N. Gomez∗†\nUniversity of Toronto\naidan@cs.toronto.edu\nŁukasz

In [70]:
type(documents[0])

langchain_core.documents.base.Document

In [80]:
def documents_chunking(documents):
    splitter = RecursiveCharacterTextSplitter(
        chunk_size=800,      # was 5000
        chunk_overlap=50,    # was 500
    )
    # basic cleanup before splitting
    for d in documents:
        d.page_content = " ".join(d.page_content.split())
    chunks = splitter.split_documents(documents)
    print(f"Processed {len(chunks)} chunks")
    return chunks
chunked_documents = documents_chunking(documents)
chunked_documents[0:3]

Processed 73 chunks


[Document(metadata={'producer': 'PyPDF2', 'creator': '', 'creationdate': '', 'source': '../data/pdf/NIPS-2017-attention-is-all-you-need-Paper.pdf', 'file_path': '../data/pdf/NIPS-2017-attention-is-all-you-need-Paper.pdf', 'total_pages': 11, 'format': 'PDF 1.3', 'title': 'Attention is All you Need', 'author': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, Illia Polosukhin', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'keywords': '', 'moddate': '2018-02-12T21:22:10-08:00', 'trapped': '', 'modDate': "D:20180212212210-08'00'", 'creationDate': '', 'page': 0}, page_content='Attention Is All You Need Ashish Vaswani∗ Google Brain avaswani@google.com Noam Shazeer∗ Google Brain noam@google.com Niki Parmar∗ Google Research nikip@google.com Jakob Uszkoreit∗ Google Research usz@google.com Llion Jones∗ Google Research llion@google.com Aidan N. Gomez∗† University of Toronto aidan@cs.toronto.edu Łukasz Kaiser∗ Google Bra

### Embeddings and VectorStoreDB

In [81]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

In [82]:
from pyexpat import model


class EmbeddingModel:
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        self.model_name = model_name
        self.model = None
        self.load_model()
    
    def load_model(self):
        self.model = SentenceTransformer(self.model_name)
        print(f"Loaded model: {self.model_name}")
        print(f"Embedding dimension: {self.model.get_sentence_embedding_dimension()}")

    def generate_embedding(self, text: List[str]) -> np.ndarray:
    #     embedding = self.model.encode(text, show_progress_bar=True)
    #     return embedding
        return self.model.encode(
            text,
            normalize_embeddings=True,   # ensures cosine distance in [0,2], similarity = dot
            show_progress_bar=True
        )
    
    
    
embedding_manager = EmbeddingModel() 
embedding_manager

Loaded model: all-MiniLM-L6-v2
Embedding dimension: 384


### Vector Store

In [83]:
import os
import uuid
from typing import List, Any, Optional

import numpy as np
import chromadb

class VectorStore:
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "./data/vector_store"):
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None          # chromadb.PersistentClient
        self.collection = None      # chroma Collection
        self.initialize_vector_store()

    def initialize_vector_store(self):
        # set up client + collection
        os.makedirs(self.persist_directory, exist_ok=True)
        self.client = chromadb.PersistentClient(path=self.persist_directory)
        self.collection = self.client.get_or_create_collection(
            name=self.collection_name,
            metadata={"description": "PDF Documents Embeddings Collection"},
        )

        print(f"Vector store initialized with collection: {self.collection_name}")
        print(f"Existing number of documents in collection: {self.collection.count()}")

    def add_documents(self, documents: List[Any], embeddings: np.ndarray, ids: Optional[List[str]] = None):
        if embeddings is None or len(documents) == 0:
            raise ValueError("documents and embeddings must be non-empty.")
        if embeddings.ndim != 2:
            raise ValueError(f"embeddings must be 2D (N, D); got shape {embeddings.shape}")
        if len(documents) != embeddings.shape[0]:
            raise ValueError("Number of documents and embeddings must be the same.")

        n = len(documents)
        ids = ids or [str(uuid.uuid4()) for _ in range(n)]
        if len(ids) != n:
            raise ValueError("Length of ids must match number of documents.")

        texts = [doc.page_content for doc in documents]
        metadatas = []
        for i, doc in enumerate(documents):
            md = dict(getattr(doc, "metadata", {}) or {})
            md["doc_index"] = i
            md["content_length"] = len(doc.page_content)
            metadatas.append(md)

        embeddings_list = embeddings.tolist()

        self.collection.add(
            ids=ids,
            documents=texts,
            metadatas=metadatas,
            embeddings=embeddings_list,
        )
        print(f"Added {n} items. Collection count: {self.collection.count()}")

vectorstore = VectorStore()
vectorstore

Vector store initialized with collection: pdf_documents
Existing number of documents in collection: 194


In [84]:
len(chunked_documents)

73

In [85]:
# ### convert text to embeddings
# texts = [doc.page_content for doc in chunked_documents]
# texts

In [86]:
# ## generate embeddings
# embeddings = embedding_manager.generate_embedding(texts)
# embeddings.shape

In [87]:
texts = [c.page_content for c in chunked_documents]
embeddings = embedding_manager.generate_embedding(texts)
print(embeddings.shape)

Batches: 100%|██████████| 3/3 [00:00<00:00,  5.73it/s]

(73, 384)


In [88]:
vectorstore.add_documents(documents=chunked_documents, embeddings=embeddings)



Added 73 items. Collection count: 267


# Data Retrieval

In [91]:
from turtle import distance



# ...existing code...
class RagRetriever:
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingModel):
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5) -> List[Dict[str, Any]]:
        print(f"Query: {query}")
        q_emb = self.embedding_manager.generate_embedding([query])[0]
        result = self.vector_store.collection.query(
            query_embeddings=[q_emb.tolist()],
            n_results=top_k,
        )
        out = []
        if result.get("documents") and result["documents"][0]:
            docs = result["documents"][0]
            metas = result["metadatas"][0]
            dists = result["distances"][0]
            ids = result["ids"][0]
            for i, (doc, md, dist, id_) in enumerate(zip(docs, metas, dists, ids)):
                # If distances are cosine distance: similarity = 1 - dist
                similarity = 1 - dist
                out.append({
                    "id": id_,
                    "similarity": similarity,
                    "distance": dist,
                    "rank": i + 1,
                    "metadata": md,
                    "text": doc[:300] + ("..." if len(doc) > 300 else "")
                })
            out.sort(key=lambda x: x["similarity"], reverse=True)
        else:
            print("No results.")
        return out

rag_retriever = RagRetriever(vector_store=vectorstore, embedding_manager=embedding_manager)
results = rag_retriever.retrieve("What is attention is all you need?")
results

Query: What is attention is all you need?


Batches: 100%|██████████| 1/1 [00:00<00:00, 29.08it/s]


[{'id': '181b156c-5b98-482d-a9db-866aa6c97d8b',
  'similarity': -0.03867638111114502,
  'distance': 1.038676381111145,
  'rank': 1,
  'metadata': {'source': '../data/pdf/NIPS-2017-attention-is-all-you-need-Paper.pdf',
   'title': 'Attention is All you Need',
   'author': 'Ashish Vaswani, Noam Shazeer, Niki Parmar, Jakob Uszkoreit, Llion Jones, Aidan N. Gomez, Łukasz Kaiser, Illia Polosukhin',
   'creationDate': '',
   'moddate': '2018-02-12T21:22:10-08:00',
   'format': 'PDF 1.3',
   'page': 6,
   'trapped': '',
   'content_length': 769,
   'modDate': "D:20180212212210-08'00'",
   'creationdate': '',
   'keywords': '',
   'doc_index': 59,
   'producer': 'PyPDF2',
   'file_path': '../data/pdf/NIPS-2017-attention-is-all-you-need-Paper.pdf',
   'creator': '',
   'subject': 'Neural Information Processing Systems http://nips.cc/',
   'total_pages': 11},
  'text': 'the approach we take in our model.\nAs side beneﬁt, self-attention could yield more interpretable models. We inspect attention d

In [66]:
from turtle import distance


# ...existing code...
class RagRetriever:
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingModel):
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0) -> List[Dict[str, Any]]:
        print(f"Retrieving top {top_k} documents for query: '{query}', score threshold: {score_threshold}")
        query_embedding = self.embedding_manager.generate_embedding([query])[0]
        results = self.vector_store.collection.query(
            query_embeddings=[query_embedding.tolist()],
            n_results=top_k,
        )
        retrieved_items = []

        if results.get('documents') and results['documents'][0]:
            documents = results['documents'][0]
            metadatas = results['metadatas'][0]
            distances = results['distances'][0]
            ids = results['ids'][0]

            for i, (doc, md, dist, id_) in enumerate(zip(documents, metadatas, distances, ids)):
                score = 1 - dist
                if score >= score_threshold:
                    retrieved_items.append({
                        "id": id_,
                        "document": doc,
                        "metadata": md,
                        "score": score,
                        "distance": dist,
                        "rank": i + 1,
                    })
                    print(f"Retrieved doc {i}: ID={id_}, Score={score:.4f}")
                else:
                    print(f"Doc {i} below threshold: ID={id_}, Score={score:.4f}")
        else:
            print("No documents retrieved.")
        return retrieved_items

rag_retriever = RagRetriever(vector_store=vectorstore, embedding_manager=embedding_manager)
rag_retriever

In [67]:
results = rag_retriever.retrieve(query="What is Recurrent models?", top_k=5, score_threshold=0)
results


Retrieving top 5 documents for query: 'What is Recurrent models?', score threshold: 0


Batches: 100%|██████████| 1/1 [00:00<00:00, 167.02it/s]

Doc 0 below threshold: ID=1694f11b-cff8-45b2-b967-10d79f99c2dc, Score=-0.0248
Doc 1 below threshold: ID=9e11e9ff-b560-4128-8c85-c3a29b5373bc, Score=-0.0284
Doc 2 below threshold: ID=3b658e14-945e-43bc-b1cf-faee3de73f6a, Score=-0.0356
Doc 3 below threshold: ID=768d9b22-b038-4717-b950-1fa85ee2983b, Score=-0.0381
Doc 4 below threshold: ID=47a6622c-bd6c-43d7-8914-2467d29a2fae, Score=-0.0525


[]

In [1]:
import os
from langchain_community.document_loaders import PyPDFLoader, PyMuPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter
from pathlib import Path
### Read all the pdf's inside the directory
def process_all_pdfs(pdf_directory):
    """Process all PDF files in a directory"""
    all_documents = []
    pdf_dir = Path(pdf_directory)
    
    # Find all PDF files recursively
    pdf_files = list(pdf_dir.glob("**/*.pdf"))
    
    print(f"Found {len(pdf_files)} PDF files to process")
    
    for pdf_file in pdf_files:
        print(f"\nProcessing: {pdf_file.name}")
        try:
            loader = PyPDFLoader(str(pdf_file))
            documents = loader.load()
            
            # Add source information to metadata
            for doc in documents:
                doc.metadata['source_file'] = pdf_file.name
                doc.metadata['file_type'] = 'pdf'
            
            all_documents.extend(documents)
            print(f"  ✓ Loaded {len(documents)} pages")
            
        except Exception as e:
            print(f"  ✗ Error: {e}")
    
    print(f"\nTotal documents loaded: {len(all_documents)}")
    return all_documents

# Process all PDFs in the data directory
all_pdf_documents = process_all_pdfs("../data/pdf")

/Users/manasvi/Github/RAG-Pipeline/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Found 2 PDF files to process

Processing: NIPS-2017-attention-is-all-you-need-Paper.pdf
  ✓ Loaded 11 pages

Processing: 2310.05421v1.pdf
  ✓ Loaded 4 pages

Total documents loaded: 15


In [2]:
all_pdf_documents

[Document(metadata={'producer': 'PyPDF2', 'creator': 'PyPDF', 'creationdate': '', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'publisher': 'Curran Associates, Inc.', 'language': 'en-US', 'created': '2017', 'eventtype': 'Poster', 'description-abstract': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks in an encoder and decoder configuration. The best performing such models also connect the encoder and decoder through an attentionm echanisms.  We propose a novel, simple network architecture based solely onan attention mechanism, dispensing with recurrence and convolutions entirely.Experiments on two machine translation tasks show these models to be superiorin quality while being more parallelizable and requiring significantly less timeto train. Our single model with 165 million parameters, achieves 27.5 BLEU onEnglish-to-German translation, improving over the existing best ensemble result by over 1 BLEU. On 

In [3]:
### Text splitting get into chunks

def split_documents(documents,chunk_size=1000,chunk_overlap=200):
    """Split documents into smaller chunks for better RAG performance"""
    text_splitter = RecursiveCharacterTextSplitter(
        chunk_size=chunk_size,
        chunk_overlap=chunk_overlap,
        length_function=len,
        separators=["\n\n", "\n", " ", ""]
    )
    split_docs = text_splitter.split_documents(documents)
    print(f"Split {len(documents)} documents into {len(split_docs)} chunks")
    
    # Show example of a chunk
    if split_docs:
        print(f"\nExample chunk:")
        print(f"Content: {split_docs[0].page_content[:200]}...")
        print(f"Metadata: {split_docs[0].metadata}")
    
    return split_docs
chunks=split_documents(all_pdf_documents)
chunks

Split 15 documents into 62 chunks

Example chunk:
Content: Attention Is All You Need
Ashish Vaswani∗
Google Brain
avaswani@google.com
Noam Shazeer∗
Google Brain
noam@google.com
Niki Parmar∗
Google Research
nikip@google.com
Jakob Uszkoreit∗
Google Research
usz...
Metadata: {'producer': 'PyPDF2', 'creator': 'PyPDF', 'creationdate': '', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'publisher': 'Curran Associates, Inc.', 'language': 'en-US', 'created': '2017', 'eventtype': 'Poster', 'description-abstract': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks in an encoder and decoder configuration. The best performing such models also connect the encoder and decoder through an attentionm echanisms.  We propose a novel, simple network architecture based solely onan attention mechanism, dispensing with recurrence and convolutions entirely.Experiments on two machine translation tasks show these models to be superiorin

[Document(metadata={'producer': 'PyPDF2', 'creator': 'PyPDF', 'creationdate': '', 'subject': 'Neural Information Processing Systems http://nips.cc/', 'publisher': 'Curran Associates, Inc.', 'language': 'en-US', 'created': '2017', 'eventtype': 'Poster', 'description-abstract': 'The dominant sequence transduction models are based on complex recurrent orconvolutional neural networks in an encoder and decoder configuration. The best performing such models also connect the encoder and decoder through an attentionm echanisms.  We propose a novel, simple network architecture based solely onan attention mechanism, dispensing with recurrence and convolutions entirely.Experiments on two machine translation tasks show these models to be superiorin quality while being more parallelizable and requiring significantly less timeto train. Our single model with 165 million parameters, achieves 27.5 BLEU onEnglish-to-German translation, improving over the existing best ensemble result by over 1 BLEU. On 

In [4]:
import numpy as np
from sentence_transformers import SentenceTransformer
import chromadb
from chromadb.config import Settings
import uuid
from typing import List, Dict, Any, Tuple
from sklearn.metrics.pairwise import cosine_similarity

class EmbeddingManager:
    """Handles document embedding generation using SentenceTransformer"""
    
    def __init__(self, model_name: str = "all-MiniLM-L6-v2"):
        """
        Initialize the embedding manager
        
        Args:
            model_name: HuggingFace model name for sentence embeddings
        """
        self.model_name = model_name
        self.model = None
        self._load_model()

    def _load_model(self):
        """Load the SentenceTransformer model"""
        try:
            print(f"Loading embedding model: {self.model_name}")
            self.model = SentenceTransformer(self.model_name)
            print(f"Model loaded successfully. Embedding dimension: {self.model.get_sentence_embedding_dimension()}")
        except Exception as e:
            print(f"Error loading model {self.model_name}: {e}")
            raise

    def generate_embeddings(self, texts: List[str]) -> np.ndarray:
        """
        Generate embeddings for a list of texts
        
        Args:
            texts: List of text strings to embed
            
        Returns:
            numpy array of embeddings with shape (len(texts), embedding_dim)
        """
        if not self.model:
            raise ValueError("Model not loaded")
        
        print(f"Generating embeddings for {len(texts)} texts...")
        embeddings = self.model.encode(texts, show_progress_bar=True)
        print(f"Generated embeddings with shape: {embeddings.shape}")
        return embeddings


## initialize the embedding manager

embedding_manager=EmbeddingManager()
embedding_manager

Loading embedding model: all-MiniLM-L6-v2
Model loaded successfully. Embedding dimension: 384


In [5]:
class VectorStore:
    """Manages document embeddings in a ChromaDB vector store"""
    
    def __init__(self, collection_name: str = "pdf_documents", persist_directory: str = "../data/vector_store"):
        """
        Initialize the vector store
        
        Args:
            collection_name: Name of the ChromaDB collection
            persist_directory: Directory to persist the vector store
        """
        self.collection_name = collection_name
        self.persist_directory = persist_directory
        self.client = None
        self.collection = None
        self._initialize_store()

    def _initialize_store(self):
        """Initialize ChromaDB client and collection"""
        try:
            # Create persistent ChromaDB client
            os.makedirs(self.persist_directory, exist_ok=True)
            self.client = chromadb.PersistentClient(path=self.persist_directory)
            
            # Get or create collection
            self.collection = self.client.get_or_create_collection(
                name=self.collection_name,
                metadata={"description": "PDF document embeddings for RAG"}
            )
            print(f"Vector store initialized. Collection: {self.collection_name}")
            print(f"Existing documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error initializing vector store: {e}")
            raise

    def add_documents(self, documents: List[Any], embeddings: np.ndarray):
        """
        Add documents and their embeddings to the vector store
        
        Args:
            documents: List of LangChain documents
            embeddings: Corresponding embeddings for the documents
        """
        if len(documents) != len(embeddings):
            raise ValueError("Number of documents must match number of embeddings")
        
        print(f"Adding {len(documents)} documents to vector store...")
        
        # Prepare data for ChromaDB
        ids = []
        metadatas = []
        documents_text = []
        embeddings_list = []
        
        for i, (doc, embedding) in enumerate(zip(documents, embeddings)):
            # Generate unique ID
            doc_id = f"doc_{uuid.uuid4().hex[:8]}_{i}"
            ids.append(doc_id)
            
            # Prepare metadata
            metadata = dict(doc.metadata)
            metadata['doc_index'] = i
            metadata['content_length'] = len(doc.page_content)
            metadatas.append(metadata)
            # Document content
            documents_text.append(doc.page_content)
            
            # Embedding
            embeddings_list.append(embedding.tolist())
        
        # Add to collection
        try:
            self.collection.add(
                ids=ids,
                embeddings=embeddings_list,
                metadatas=metadatas,
                documents=documents_text
            )
            print(f"Successfully added {len(documents)} documents to vector store")
            print(f"Total documents in collection: {self.collection.count()}")
            
        except Exception as e:
            print(f"Error adding documents to vector store: {e}")
            raise

vectorstore=VectorStore()
vectorstore

Vector store initialized. Collection: pdf_documents
Existing documents in collection: 62


In [6]:
### Convert the text to embeddings
texts=[doc.page_content for doc in chunks]

## Generate the Embeddings

embeddings=embedding_manager.generate_embeddings(texts)

##store int he vector dtaabase
vectorstore.add_documents(chunks,embeddings)

Generating embeddings for 62 texts...


Batches: 100%|██████████| 2/2 [00:00<00:00,  8.48it/s]

Generated embeddings with shape: (62, 384)
Adding 62 documents to vector store...
Successfully added 62 documents to vector store
Total documents in collection: 124


In [7]:
class RAGRetriever:
    """Handles query-based retrieval from the vector store"""
    
    def __init__(self, vector_store: VectorStore, embedding_manager: EmbeddingManager):
        """
        Initialize the retriever
        
        Args:
            vector_store: Vector store containing document embeddings
            embedding_manager: Manager for generating query embeddings
        """
        self.vector_store = vector_store
        self.embedding_manager = embedding_manager

    def retrieve(self, query: str, top_k: int = 5, score_threshold: float = 0.0) -> List[Dict[str, Any]]:
        """
        Retrieve relevant documents for a query
        
        Args:
            query: The search query
            top_k: Number of top results to return
            score_threshold: Minimum similarity score threshold
            
        Returns:
            List of dictionaries containing retrieved documents and metadata
        """
        print(f"Retrieving documents for query: '{query}'")
        print(f"Top K: {top_k}, Score threshold: {score_threshold}")
        
        # Generate query embedding
        query_embedding = self.embedding_manager.generate_embeddings([query])[0]
        
        # Search in vector store
        try:
            results = self.vector_store.collection.query(
                query_embeddings=[query_embedding.tolist()],
                n_results=top_k
            )
            
            # Process results
            retrieved_docs = []
            
            if results['documents'] and results['documents'][0]:
                documents = results['documents'][0]
                metadatas = results['metadatas'][0]
                distances = results['distances'][0]
                ids = results['ids'][0]
                
                for i, (doc_id, document, metadata, distance) in enumerate(zip(ids, documents, metadatas, distances)):
                    # Convert distance to similarity score (ChromaDB uses cosine distance)
                    similarity_score = 1 - distance
                    
                    if similarity_score >= score_threshold:
                        retrieved_docs.append({
                            'id': doc_id,
                            'content': document,
                            'metadata': metadata,
                            'similarity_score': similarity_score,
                            'distance': distance,
                            'rank': i + 1
                        })
                
                print(f"Retrieved {len(retrieved_docs)} documents (after filtering)")
            else:
                print("No documents found")
            
            return retrieved_docs
            
        except Exception as e:
            print(f"Error during retrieval: {e}")
            return []

rag_retriever=RAGRetriever(vectorstore,embedding_manager)

In [8]:
rag_retriever.retrieve("What is attention is all you need")


Retrieving documents for query: 'What is attention is all you need'
Top K: 5, Score threshold: 0.0
Generating embeddings for 1 texts...


Batches: 100%|██████████| 1/1 [00:00<00:00, 20.05it/s]

Generated embeddings with shape: (1, 384)
Retrieved 0 documents (after filtering)


[]